Models that do not work because they use sigma_signma

In [ ]:
def lnlikelin(theta, x, y,yerr):# modelo por cuadratura, no pasa PP check
    m ,b = theta
    model = m * x**2 + b**2
    inv_sigma2 = 1.0/(yerr**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(2*np.pi*inv_sigma2)))
def lnpriorlin(theta):
    m, b = theta
    if 0 < m < 10 and 0 < b < 50:
        return 0.0
    return -np.inf
# Posterior probability
def lnproblin(theta, x, y,yerr):
    lp = lnpriorlin(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlikelin(theta, x, y,yerr)

def discrepq(samples,x,y,yerr):
    discs=[]
    disco=[]
    rej=0
    ndisc=len(samples)//100
    for i in range(ndisc):
        yexp=samples[i,0]*x**2+samples[i,1]**2
        modpar=samples[np.random.randint(len(samples))]
        ysim=np.random.normal(modpar[0]*x**2+modpar[1]**2,yerr)
        if np.any(ysim<0)|np.any(ysim<0):
            rej+=1
        else:        
#            yexp=np.sqrt(yexp)
#            ysim=np.sqrt(ysim)
#            y=np.sqrt(y)
#            discs+=[((ysim-yexp)**2).sum()]
#            disco+=[((y-yexp)**2).sum()]
            discs+=[((np.sqrt(ysim)-np.sqrt(yexp))**2).sum()]
            disco+=[((np.sqrt(y)-np.sqrt(yexp))**2).sum()]
    print("Rejection rate =",100*rej//ndisc,"%") # rejection rate
    if 100*rej//ndisc < 100:
        discs=np.array(discs)
        disco=np.array(disco)
        plt.figure(figsize=(6,6))
        plt.scatter(discs,disco)
        dmax=np.max(np.hstack((disco,discs)))
        ppp=np.linspace(0,dmax,2)
        plt.plot(ppp,ppp,c='r')
        plt.xlim(0,dmax)
        plt.ylim(0,dmax)
        print("Bayesian p-value:", (disco<discs).sum()*1./len(disco))
        
def discrepq2(samples,x,y,yerr):
    discs=[]
    disco=[]
    rej=0
    ndisc=len(samples)//100
    for i in range(ndisc):
        yexp=samples[i,0]*x**2+samples[i,1]**2
        modpar=samples[np.random.randint(len(samples))]
        ysim=np.random.normal(modpar[0]*x**2+modpar[1]**2,yerr)
        if np.any(ysim<0)|np.any(ysim<0):
            rej+=1
        else:        
            yexp=np.sqrt(yexp)
            ysim=np.sqrt(ysim)
            y=np.sqrt(y)
#            discs+=[((ysim-yexp)**2).sum()]
#            disco+=[((y-yexp)**2).sum()]
            discs+=[((np.sqrt(ysim)-np.sqrt(yexp))**2).sum()]
            disco+=[((np.sqrt(y)-np.sqrt(yexp))**2).sum()]
    print("Rejection rate =",100*rej//ndisc,"%") # rejection rate
    if 100*rej//ndisc < 100:
        discs=np.array(discs)
        disco=np.array(disco)
        plt.figure(figsize=(6,6))
        plt.scatter(discs,disco)
        dmax=np.max(np.hstack((disco,discs)))
        ppp=np.linspace(0,dmax,2)
        plt.plot(ppp,ppp,c='r')
        plt.xlim(0,dmax)
        plt.ylim(0,dmax)
        print("Bayesian p-value:", (disco<discs).sum()*1./len(disco))

In [ ]:
# Y este modelo como pa' que?
# modelo por cuadratura con w=yerr**-1 no funciona con FT p-value
# modelo por cuadratura con w=yerr**-2 no funciona con FT p-value (aun menos)
# modelo por cuadratura con w=yerr**-1 no funciona con FT p-value usando directamente y
# modelo por cuadratura con w=yerr**-2 no funciona con FT p-value usando directamente y (aun menos)

In [ ]:
# en este punto no es necesario graficar los resultados del mcmc, pero valdría la pena dejarlo correr mas? 
# tambien se podrian pesar cada samples, pero no pienso que sea correcto
bdist=np.array([])
mdist=np.array([])
coun=0
for i in range(1,26): # sin el yerr al cuadrado da buenos resultados para el errsist, hay que comparar los y_sim
    filt=np.array(em)>i
    x=np.array(dboot)[filt]
    y=np.array(bootp50)[filt]
    #yerr=np.array(wstd)[filt] # wstd
    yerr=np.array(bootsig)[filt]
    y=y**2
    yerr=yerr**2
#    nll = lambda *args: -lnlikelin(*args)
#    result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
    result = (1,4)
    m_ml, b_ml = result
#    if b_ml > 0:
    ndim, nwalkers = len(result), 100
    pos = [result + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnproblin, args=(x, y,yerr)) 
    # 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
    sampler.run_mcmc(pos, 1000);
    # Here we remove the burn-in steps, which we estimate to be around 100. 
    # This can be estimated by calculating the autocorrelation time (see below)
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
    bdist=np.hstack((bdist,samples[:,1]))
    mdist=np.hstack((mdist,samples[:,0]))
    print(i,samples.mean(axis=0),filt.sum(),x.max())
    coun+=1
    discrepq(samples,x,y,yerr)

In [ ]:
1 [ 0.08177229  0.26353757] 8792 837.038654193
Rejection rate = 100 %
2 [ 0.10086809  0.26376468] 7206 837.038654193
Rejection rate = 100 %
3 [ 0.12185908  0.19736986] 5234 837.038654193
Rejection rate = 100 %
4 [ 0.12912342  0.16026281] 3843 837.038654193
Rejection rate = 100 %
5 [ 0.1275923   0.16227569] 3201 185.046755778
Rejection rate = 100 %
6 [ 0.1294664   0.14798133] 2742 174.444324283
Rejection rate = 100 %
7 [ 0.13117321  0.14815377] 2299 169.931786108
Rejection rate = 100 %
8 [ 0.13187017  0.13495028] 1827 169.931786108
Rejection rate = 100 %
9 [ 0.13217861  0.13647739] 1489 143.188488944
Rejection rate = 100 %
10 [ 0.1315867   0.13280771] 1263 143.188488944
Rejection rate = 100 %
11 [ 0.12483849  1.70375826] 1061 143.188488944
Rejection rate = 99 %
Bayesian p-value: 0.0
12 [ 0.12057113  1.98913581] 826 143.188488944
Rejection rate = 99 %
Bayesian p-value: 0.0
13 [ 0.11570399  2.34740868] 602 143.188488944
Rejection rate = 95 %
Bayesian p-value: 0.0
14 [ 0.11818783  2.91739989] 468 143.188488944
Rejection rate = 75 %
Bayesian p-value: 0.0
15 [ 0.12599279  2.76178996] 367 143.188488944
Rejection rate = 46 %
Bayesian p-value: 0.0
16 [ 0.13983217  2.51039268] 286 143.188488944
Rejection rate = 22 %
Bayesian p-value: 0.0
17 [ 0.13462393  2.63826754] 231 132.149450738
Rejection rate = 19 %
Bayesian p-value: 0.0
18 [ 0.12897396  3.06544774] 179 132.149450738
Rejection rate = 16 %
Bayesian p-value: 0.0
19 [ 0.12030781  3.16717815] 132 132.149450738
Rejection rate = 11 %
Bayesian p-value: 0.0
20 [ 0.1151967   3.16860286] 100 132.149450738
Rejection rate = 10 %
Bayesian p-value: 0.0
21 [ 0.11963383  3.77526148] 75 95.1061379085
Rejection rate = 6 %
Bayesian p-value: 0.0
22 [ 0.23627729  2.05045446] 53 65.6827536324
Rejection rate = 0 %
Bayesian p-value: 0.0
23 [ 0.23879858  0.62918635] 31 31.6677431289
Rejection rate = 1 %
Bayesian p-value: 0.0
24 [ 0.23931359  0.96555821] 22 31.6677431289
Rejection rate = 1 %
Bayesian p-value: 0.0
25 [ 0.19342843  2.87930209] 15 28.9241085243
Rejection rate = 1 %
Bayesian p-value: 0.0

In [ ]:
# en este punto no es necesario graficar los resultados del mcmc, pero valdría la pena dejarlo correr mas? 
# tambien se podrian pesar cada samples, pero no pienso que sea correcto
bdist=np.array([])
mdist=np.array([])
coun=0
for i in range(1,26): # sin el yerr al cuadrado da buenos resultados para el errsist, hay que comparar los y_sim
    filt=np.array(em)>i
    x=np.array(dboot)[filt]
    y=np.array(bootp50)[filt]
    #yerr=np.array(wstd)[filt] # wstd
    yerr=np.array(bootsig)[filt]
    y=y**2
    yerr=yerr**2
#    nll = lambda *args: -lnlikelin(*args)
#    result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
    result = (1,4)
    m_ml, b_ml = result
#    if b_ml > 0:
    ndim, nwalkers = len(result), 100
    pos = [result + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnproblin, args=(x, y,yerr)) 
    # 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
    sampler.run_mcmc(pos, 1000);
    # Here we remove the burn-in steps, which we estimate to be around 100. 
    # This can be estimated by calculating the autocorrelation time (see below)
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
    bdist=np.hstack((bdist,samples[:,1]))
    mdist=np.hstack((mdist,samples[:,0]))
    print(i,samples.mean(axis=0),filt.sum(),x.max())
    coun+=1
    discrepq2(samples,x,y,yerr)

In [ ]:
1 [ 0.08178792  0.25867817] 8792 837.038654193
Rejection rate = 100 %
2 [ 0.10086335  0.26130766] 7206 837.038654193
Rejection rate = 100 %
3 [ 0.12184469  0.19176497] 5234 837.038654193
Rejection rate = 100 %
4 [ 0.12908163  0.15559771] 3843 837.038654193
Rejection rate = 100 %
5 [ 0.1276663   0.16255627] 3201 185.046755778
Rejection rate = 100 %
6 [ 0.12953682  0.14794907] 2742 174.444324283
Rejection rate = 100 %
7 [ 0.13120352  0.14239694] 2299 169.931786108
Rejection rate = 100 %
8 [ 0.13193068  0.13442817] 1827 169.931786108
Rejection rate = 100 %
9 [ 0.13208565  0.13060815] 1489 143.188488944
Rejection rate = 100 %
10 [ 0.13154495  0.13060121] 1263 143.188488944
Rejection rate = 100 %
11 [ 0.12490116  1.70160008] 1061 143.188488944
Rejection rate = 99 %
Bayesian p-value: 0.0
12 [ 0.12053229  1.99367954] 826 143.188488944
Rejection rate = 99 %
Bayesian p-value: 0.0
13 [ 0.11567324  2.35016154] 602 143.188488944
Rejection rate = 95 %
Bayesian p-value: 0.0
14 [ 0.11820935  2.91633289] 468 143.188488944
Rejection rate = 76 %
Bayesian p-value: 0.0
15 [ 0.12598781  2.76260258] 367 143.188488944
Rejection rate = 48 %
Bayesian p-value: 0.0
16 [ 0.13992049  2.50666231] 286 143.188488944
Rejection rate = 22 %
Bayesian p-value: 0.0
17 [ 0.13469834  2.63714625] 231 132.149450738
Rejection rate = 18 %
Bayesian p-value: 0.0
18 [ 0.12891659  3.06528637] 179 132.149450738
Rejection rate = 13 %
Bayesian p-value: 0.0
19 [ 0.12026418  3.1657758 ] 132 132.149450738
Rejection rate = 10 %
Bayesian p-value: 0.0
20 [ 0.11519168  3.17254392] 100 132.149450738
Rejection rate = 11 %
Bayesian p-value: 0.0
21 [ 0.11966976  3.77763862] 75 95.1061379085
Rejection rate = 6 %
Bayesian p-value: 0.0
22 [ 0.23635292  2.04494741] 53 65.6827536324
Rejection rate = 1 %
Bayesian p-value: 0.0
23 [ 0.23855112  0.65309217] 31 31.6677431289
Rejection rate = 1 %
Bayesian p-value: 0.0
24 [ 0.23933481  0.98227527] 22 31.6677431289
Rejection rate = 1 %
Bayesian p-value: 0.0
25 [ 0.1923731   2.93558308] 15 28.9241085243
Rejection rate = 1 %
Bayesian p-value: 0.0

In [ ]:
# en este punto no es necesario graficar los resultados del mcmc, pero valdría la pena dejarlo correr mas? 
# tambien se podrian pesar cada samples, pero no pienso que sea correcto
bdist=np.array([])
mdist=np.array([])
coun=0
for i in range(1,26): # sin el yerr al cuadrado da buenos resultados para el errsist, hay que comparar los y_sim
    filt=np.array(em)>i
    filt.sum()
    x=np.array(dboot)[filt]
    y=np.array(bootp50)[filt]
    #yerr=np.array(wstd)[filt] # wstd
    yerr=np.array(bootsig)[filt]
    y=y**2
#    yerr=yerr**2
#    nll = lambda *args: -lnlikelin(*args)
#    result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
    result = (1,4)
    m_ml, b_ml = result
#    if b_ml > 0:
    ndim, nwalkers = len(result), 100
    pos = [result + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnproblin, args=(x, y,yerr)) 
    # 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
    sampler.run_mcmc(pos, 1000);
    # Here we remove the burn-in steps, which we estimate to be around 100. 
    # This can be estimated by calculating the autocorrelation time (see below)
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
    bdist=np.hstack((bdist,samples[:,1]))
    mdist=np.hstack((mdist,samples[:,0]))
    print(i,samples.mean(axis=0),filt.sum(),x.max())
    coun+=1
    discrepq(samples,x,y,yerr)

In [ ]:
1 [ 0.13014661  0.02249417] 8792 837.038654193
Rejection rate = 98 %
Bayesian p-value: 0.0
2 [ 0.15043282  0.02330042] 7206 837.038654193
Rejection rate = 85 %
Bayesian p-value: 0.0
3 [ 0.27755731  0.01909256] 5234 837.038654193
Rejection rate = 36 %
Bayesian p-value: 0.0
4 [ 0.37922334  0.01066219] 3843 837.038654193
Rejection rate = 14 %
Bayesian p-value: 0.0
5 [ 0.11723756  3.70622824] 3201 185.046755778
Rejection rate = 0 %
Bayesian p-value: 0.0
6 [ 0.12090294  3.60805821] 2742 174.444324283
Rejection rate = 0 %
Bayesian p-value: 0.0
7 [ 0.12520527  3.479122  ] 2299 169.931786108
Rejection rate = 0 %
Bayesian p-value: 0.0
8 [ 0.12647446  3.43288062] 1827 169.931786108
Rejection rate = 0 %
Bayesian p-value: 0.0
9 [ 0.12894976  3.24267   ] 1489 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
10 [ 0.12917684  3.17459488] 1263 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
11 [ 0.12302885  4.99338345] 1061 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
12 [ 0.11705312  5.25616591] 826 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
13 [ 0.10668144  5.58731701] 602 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
14 [ 0.10790672  5.85403389] 468 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
15 [ 0.11391691  5.59125219] 367 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
16 [ 0.11749045  5.43884298] 286 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
17 [ 0.11381856  5.42952379] 231 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
18 [ 0.10817756  5.70681869] 179 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
19 [ 0.09269254  5.92803162] 132 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
20 [ 0.07757713  6.18690058] 100 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
21 [ 0.07874387  6.4775217 ] 75 95.1061379085
Rejection rate = 0 %
Bayesian p-value: 0.0

/home/ecci/miniconda3/envs/py3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)

22 [ 0.19156265  4.26534566] 53 65.6827536324
Rejection rate = 0 %
Bayesian p-value: 0.0
23 [ 0.28089544  0.16434704] 31 31.6677431289
Rejection rate = 0 %
Bayesian p-value: 0.0
24 [ 0.28739079  0.20305322] 22 31.6677431289
Rejection rate = 0 %
Bayesian p-value: 0.0
25 [ 0.2185522   3.52904739] 15 28.9241085243
Rejection rate = 0 %
Bayesian p-value: 0.0


In [ ]:
# en este punto no es necesario graficar los resultados del mcmc, pero valdría la pena dejarlo correr mas? 
# tambien se podrian pesar cada samples, pero no pienso que sea correcto
bdist=np.array([])
mdist=np.array([])
coun=0
for i in range(1,26): # sin el yerr al cuadrado da buenos resultados para el errsist, hay que comparar los y_sim
    filt=np.array(em)>i
    x=np.array(dboot)[filt]
    y=np.array(bootp50)[filt]
    #yerr=np.array(wstd)[filt] # wstd
    yerr=np.array(bootsig)[filt]
    y=y**2
#    yerr=yerr**2
#    nll = lambda *args: -lnlikelin(*args)
#    result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
    result = (1,4)
    m_ml, b_ml = result
#    if b_ml > 0:
    ndim, nwalkers = len(result), 100
    pos = [result + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnproblin, args=(x, y,yerr)) 
    # 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
    sampler.run_mcmc(pos, 1000);
    # Here we remove the burn-in steps, which we estimate to be around 100. 
    # This can be estimated by calculating the autocorrelation time (see below)
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
    bdist=np.hstack((bdist,samples[:,1]))
    mdist=np.hstack((mdist,samples[:,0]))
    print(i,samples.mean(axis=0),filt.sum(),x.max())
    coun+=1
    discrepq2(samples,x,y,yerr)

In [ ]:
1 [ 0.13014327  0.02456933] 8792 837.038654193
Rejection rate = 98 %
Bayesian p-value: 0.0
2 [ 0.15043434  0.02151066] 7206 837.038654193
Rejection rate = 85 %
Bayesian p-value: 0.0
3 [ 0.27756952  0.01813189] 5234 837.038654193
Rejection rate = 39 %
Bayesian p-value: 0.0
4 [ 0.37923375  0.01217702] 3843 837.038654193
Rejection rate = 15 %
Bayesian p-value: 0.0
5 [ 0.1172364  3.7078539] 3201 185.046755778
Rejection rate = 0 %
Bayesian p-value: 0.0
6 [ 0.1209095   3.60527027] 2742 174.444324283
Rejection rate = 0 %
Bayesian p-value: 0.0
7 [ 0.12520399  3.47789636] 2299 169.931786108
Rejection rate = 0 %
Bayesian p-value: 0.0
8 [ 0.1264853   3.43600496] 1827 169.931786108
Rejection rate = 0 %
Bayesian p-value: 0.0
9 [ 0.12894309  3.24300219] 1489 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
10 [ 0.1291908   3.17659175] 1263 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
11 [ 0.12302754  4.99205685] 1061 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
12 [ 0.11705771  5.25764534] 826 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
13 [ 0.1066884   5.58708309] 602 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
14 [ 0.10788931  5.85550632] 468 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
15 [ 0.11390288  5.59175737] 367 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
16 [ 0.11749313  5.43827485] 286 143.188488944
Rejection rate = 0 %
Bayesian p-value: 0.0
17 [ 0.11383392  5.42752913] 231 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
18 [ 0.10818296  5.70700978] 179 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
19 [ 0.09265667  5.92963695] 132 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
20 [ 0.07757054  6.18359378] 100 132.149450738
Rejection rate = 0 %
Bayesian p-value: 0.0
21 [ 0.0787722   6.47687316] 75 95.1061379085
Rejection rate = 0 %
Bayesian p-value: 0.0

/home/ecci/miniconda3/envs/py3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)

22 [ 0.19157658  4.26257286] 53 65.6827536324
Rejection rate = 0 %
Bayesian p-value: 0.0
23 [ 0.28087443  0.16404701] 31 31.6677431289
Rejection rate = 0 %
Bayesian p-value: 0.0
24 [ 0.28736922  0.20610256] 22 31.6677431289
Rejection rate = 0 %
Bayesian p-value: 0.0
25 [ 0.21862089  3.52855932] 15 28.9241085243
Rejection rate = 0 %
Bayesian p-value: 0.0


In [ ]:
# en este punto no es necesario graficar los resultados del mcmc, pero valdría la pena dejarlo correr mas? 
# tambien se podrian pesar cada samples, pero no pienso que sea correcto
bdist=np.array([])
mdist=np.array([])
coun=0
for i in range(15,26): # sin el yerr al cuadrado da buenos resultados para el errsist, hay que comparar los y_sim
    filt=np.array(em)>i
    filt.sum()
    x=np.array(dboot)[filt]
    y=np.array(bootp50)[filt]
    #yerr=np.array(wstd)[filt] # wstd
    yerr=np.array(bootsig)[filt]
#    y=y**2
#    yerr=yerr**2
#    nll = lambda *args: -lnlikelin(*args)
#    result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
    m_ml, b_ml = (1,4)
#    if b_ml > 0:
    ndim, nwalkers = len(result["x"]), 100
    pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob1, args=(x, y,yerr)) 
    # 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
    sampler.run_mcmc(pos, 1000);
    # Here we remove the burn-in steps, which we estimate to be around 100. 
    # This can be estimated by calculating the autocorrelation time (see below)
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
    bdist=np.hstack((bdist,samples[:,1]))
    mdist=np.hstack((mdist,samples[:,0]))
    print(i,samples.mean(axis=0))
    coun+=1
    discrep1(samples,x,y,yerr)

In [ ]:
15 [ 0.00412615  2.38598114]
Rejection rate = 100 %
16 [ 0.00410427  2.2741868 ]
Rejection rate = 100 %
17 [ 0.00414484  2.32310752]
Rejection rate = 100 %
18 [ 0.00380702  2.41682072]
Rejection rate = 100 %
19 [ 0.00351205  2.40932232]
Rejection rate = 100 %
20 [ 0.00324021  2.40094836]
Rejection rate = 100 %
21 [ 0.0035776   2.47626619]
Rejection rate = 100 %
22 [ 0.00905003  2.07935452]
Rejection rate = 100 %
23 [ 0.01637604  1.79765069]
Rejection rate = 99 %
Bayesian p-value: 0.0
24 [ 0.01618641  1.78326413]
Rejection rate = 99 %
Bayesian p-value: 0.0
25 [ 0.01325181  1.36977426]


In [ ]:
def discrep(samples,x,y,yerr):
    discs=[]
    disco=[]
    rej=0
    ndisc=len(samples)//10
    for i in range(ndisc):
        yexp=samples[i,0]*x**2+samples[i,1]**2
        modpar=samples[np.random.randint(len(samples))]
        ysim=np.random.normal(modpar[0]*x**2+modpar[1]**2,yerr)
        if np.any(ysim<0)|np.any(ysim<0):
            rej+=1
        else:        
            yexp=np.sqrt(yexp)
            ysim=np.sqrt(ysim)
            y=np.sqrt(y)
            discs+=[((np.sqrt(ysim)-np.sqrt(yexp))**2).sum()]
            disco+=[((np.sqrt(y)-np.sqrt(yexp))**2).sum()]
    print("Rejection rate =",100*rej//ndisc,"%") # rejection rate
    if 100*rej//ndisc < 100:
        discs=np.array(discs)
        disco=np.array(disco)
        plt.figure(figsize=(6,6))
        plt.scatter(discs,disco)
        dmax=np.max(np.hstack((disco,discs)))
        ppp=np.linspace(0,dmax,2)
        plt.plot(ppp,ppp,c='r')
        plt.xlim(0,dmax)
        plt.ylim(0,dmax)
        print("Bayesian p-value:", (disco<discs).sum()*1./len(disco))

In [ ]:
def lnlike1(theta, x, y,yerr):# modelo lineal (D,wstd) independiente de sigma_sigma
    m ,b = theta
    model = m * x + b
    inv_sigma2 = 1.0/(yerr**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(2*np.pi*inv_sigma2)))
def lnprior1(theta):
    m, b = theta
    if -10 < m < 10 and -50 < b < 50:
        return 0.0
    return -np.inf
# Posterior probability
def lnprob1(theta, x, y,yerr):
    lp = lnprior1(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike1(theta, x, y,yerr)

In [ ]:
def discrep1(samples,x,y,yerr):
    discs=[]
    disco=[]
    rej=0
    ndisc=len(samples)//10
    for i in range(ndisc):
        yexp=samples[i,0]*x+samples[i,1]
        modpar=samples[np.random.randint(len(samples))]
        ysim=np.random.normal(modpar[0]*x+modpar[1],yerr)
        if np.any(ysim<0)|np.any(ysim<0):
            rej+=1
        else:        
#            yexp=np.sqrt(yexp)
#            ysim=np.sqrt(ysim)
#            y=np.sqrt(y)
            discs+=[((np.sqrt(ysim)-np.sqrt(yexp))**2).sum()]
            disco+=[((np.sqrt(y)-np.sqrt(yexp))**2).sum()]
    print("Rejection rate =",100*rej//ndisc,"%") # rejection rate
    if 100*rej//ndisc < 100:
        discs=np.array(discs)
        disco=np.array(disco)
        plt.figure(figsize=(6,6))
        plt.scatter(discs,disco)
        dmax=np.max(np.hstack((disco,discs)))
        ppp=np.linspace(0,dmax,2)
        plt.plot(ppp,ppp,c='r')
        plt.xlim(0,dmax)
        plt.ylim(0,dmax)
        print("Bayesian p-value:", (disco<discs).sum()*1./len(disco))

In [ ]:
i=26
filt=np.array(em)>i
filt.sum()
x=np.array(dboot)[filt]
y=np.array(bootp50)[filt]
yerr=np.array(bootsig)[filt]
#    y=y**2
#    yerr=yerr**2
nll = lambda *args: -lnlike1(*args)
result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
m_ml, b_ml = result["x"]
print(i,m_ml,b_ml)
#    if b_ml > 0:
ndim, nwalkers = len(result["x"]), 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob1, args=(x, y,yerr)) 
# 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
sampler.run_mcmc(pos, 1000);
# Here we remove the burn-in steps, which we estimate to be around 100. 
# This can be estimated by calculating the autocorrelation time (see below)
samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
bdist=np.hstack((bdist,samples[:,1]))
mdist=np.hstack((mdist,samples[:,0]))
sth=samples.mean(axis=0)
fig = corner.corner(samples, labels=["$m$", "$b$"],truths=sth, quantiles=[0.16, 0.5, 0.84], show_titles=True,title_fmt='.4f')
print(i,sth)
coun+=1
discrep1(samples,x,y,yerr)

In [ ]:
i=26
filt=np.array(em)>i
filt.sum()
x=np.array(dboot)[filt]
y=np.array(bootp50)[filt]
yerr=np.array(bootsig)[filt]
#    y=y**2
#    yerr=yerr**2
nll = lambda *args: -lnlike1(*args)
result = op.minimize(nll, [1.]*2, args=(x, y,yerr))
m_ml, b_ml = result["x"]
print(i,m_ml,b_ml)
#    if b_ml > 0:
ndim, nwalkers = len(result["x"]), 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob1, args=(x, y,yerr)) 
# 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
sampler.run_mcmc(pos, 1000);
# Here we remove the burn-in steps, which we estimate to be around 100. 
# This can be estimated by calculating the autocorrelation time (see below)
samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
bdist=np.hstack((bdist,samples[:,1]))
mdist=np.hstack((mdist,samples[:,0]))
sth=samples.mean(axis=0)
fig = corner.corner(samples, labels=["$m$", "$b$"],truths=sth, quantiles=[0.16, 0.5, 0.84], show_titles=True,title_fmt='.4f')
print(i,sth)
coun+=1
discrep1(samples,x,y,yerr)

## Quadratic model

Works for n_meas > 22 (53 galaxies, 1219 measurements).

$$\sigma_B=mD^2+b^2 $$
$$\sigma_{SW}=f\sigma_B$$

We prefer the linear model because it uses more data. Discrepancy plot looks better, though.

In [ ]:
def lnlike3(theta, x, y):# modelo que usa una fracción de sigma_sigma
    m ,b, f = theta
    model = m * x**2+ b**2
    inv_sigma2 = 1.0/((f*model)**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(2*np.pi*inv_sigma2)))
def lnprior3(theta):
    m, b, f = theta
    if 0 < m < 10 and 0 < b < 10 and 0 < f < 1:
        return 0.0
    return -np.inf
# Posterior probability
def lnprob3(theta, x, y):
    lp = lnprior3(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike3(theta, x, y)

def discrep3(samples,x,y):
    discs=[]
    disco=[]
    rej=0
    ndisc=len(samples)//100
    for i in range(ndisc):
        yexp=samples[i,0]*x**2+samples[i,1]**2
        modpar=samples[np.random.randint(len(samples))]
        ymod=modpar[0]*x**2+modpar[1]**2
        ysim=np.random.normal(ymod,modpar[2]*ymod)
        if np.any(ysim<0)|np.any(ysim<0):
            rej+=1
        else:        
#            yexp=np.sqrt(yexp)
#            ysim=np.sqrt(ysim)
#            y=np.sqrt(y)
#            discs+=[((ysim-yexp)**2).sum()]
#            disco+=[((y-yexp)**2).sum()]
            discs+=[((np.sqrt(ysim)-np.sqrt(yexp))**2).sum()]
            disco+=[((np.sqrt(y)-np.sqrt(yexp))**2).sum()]
    print("Rejection rate =",100*rej//ndisc,"%") # rejection rate
    if 100*rej//ndisc < 100:
        discs=np.array(discs)
        disco=np.array(disco)
        plt.figure(figsize=(6,6))
        plt.scatter(discs,disco)
        dmax=np.max(np.hstack((disco,discs)))
        dmin=np.min(np.hstack((disco,discs)))
        ppp=np.linspace(0,dmax,2)
        plt.plot(ppp,ppp,c='r')
        plt.xlim(dmin,dmax)
        plt.ylim(dmin,dmax)
        print("Bayesian p-value:", (disco<discs).sum()*1./len(disco))

In [ ]:
i=12 # 12 seems to be the magic number
filt=np.array(em)>i
print(filt.sum())
x=np.array(dboot)[filt]
y=np.array(bootp50)[filt]
y=np.array(bootmad)[filt]
nll = lambda *args: -lnlike3(*args)## AQUI
result = op.minimize(nll, [1.]*3, args=(x, y)) ## AQUI
m_ml, b_ml,f_ml = result["x"]## AQUI
print(i,m_ml,b_ml,f_ml)## AQUI
#    if b_ml > 0:
ndim, nwalkers = len(result["x"]), 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob3, args=(x, y)) ## AQUI
# 2. Let the walkers walk 1000 steps (10000 will do the trick as well)
sampler.run_mcmc(pos, 1000);
# Here we remove the burn-in steps, which we estimate to be around 100. 
# This can be estimated by calculating the autocorrelation time (see below)
samples = sampler.chain[:, 100:, :].reshape((-1, ndim)) 
sth=samples.mean(axis=0)
fig = corner.corner(samples, labels=["$m$", "$b$","$f$"],truths=sth, quantiles=[0.16, 0.5, 0.84], show_titles=True,title_fmt='.4f')
print(i,sth)
discrep3(samples,x,y)

In [ ]:
modpar=samples[np.random.randint(len(samples))]
ysim=np.random.normal(modpar[0]*x**2+modpar[1]**2,modpar[2]*(modpar[0]*x**2+modpar[1]**2))
plt.scatter(x,y)
plt.scatter([0],[0])
plt.scatter(x,ysim)